In [12]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
# Import the requests library.
import requests
# Import the API key.
from config import weather_api_key
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime

In [3]:
# Create a set of 2,000 random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs


In [4]:
 # Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [5]:
# Use the print() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

39.315034615227546 104.15637881409691
-62.57652216631338 67.25012260494387
-72.40371570896525 93.58911465149629
-62.49222925020268 -85.129046552692
80.8132495955272 -118.81796515005182
82.80260118791196 -82.60207812549484
82.90761491199592 50.70888336546744
84.5061272063887 -91.25838773073849
77.89215901078273 93.34064667044566
-2.4133673560943834 -57.781048882407234
56.36351151537599 163.81684848020973
-40.34444848208472 -17.018395350327722
-33.54391867935345 9.017933090700097
-46.28548791344621 127.47749039003503
-62.35545443586958 35.23181309519987
85.2082210477198 -162.95446360961012
-30.09433968117304 50.74346106641161
16.72861566637836 -41.05341251305916
-67.57873059627207 87.5730206318708
-62.25283369105152 -109.59838875197919
-40.75345479551011 71.89772485567946
72.70869301064289 -167.95436735188906
84.33833273434539 137.49114251191634
-84.11270772108871 -13.949648837986928
47.08449459478555 98.23084229792477
29.77222121198868 36.52183904639736
81.24891232259623 -101.7498416662

-53.74762360668708 -62.947642377397784
-86.42047013868155 5.5802438510513355
81.71479458083047 -105.35543007094074
0.060062259774625204 -123.0688690620658
85.90145867225283 86.33392961493485
-24.57809854202435 -85.96147099830408
-72.22003593456688 86.75817053312676
-66.90459749324857 130.96420824771758
47.856684567897844 58.74647004084417
4.5330858420233255 93.31007583518499
-0.21597592853127878 142.40530722424307
89.66106323477226 -159.52292234352905
45.48684227069799 151.26980157284186
-65.80706046258402 20.84086509614133
-52.50701744105802 -68.85033089050545
84.67125240248163 28.83218486192868
-67.93831068517456 79.06093000039482
69.36880430162773 -76.6634500762828
60.36188371053365 -161.12133619175597
41.50207280366945 -4.26377282469511
-53.77443133812023 -17.63393268919532
11.784574560023046 -65.7761196732024
22.105424991597587 -158.97416233685806
35.24484383859094 85.46353599341847
20.187731249699013 70.38945028733656
78.60378997479236 -81.86798518663699
-84.85953897329273 150.96

In [10]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)


746

In [13]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)


http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=d49fafd11cc42c8de5b4c086c421e9fe


In [18]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        description = city_weather['weather'][0]['description']
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date,
                          "Current Description": description 
                         })

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | wuwei
Processing Record 2 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 3 of Set 1 | busselton
Processing Record 4 of Set 1 | punta arenas
Processing Record 5 of Set 1 | norman wells
Processing Record 6 of Set 1 | qaanaaq
Processing Record 7 of Set 1 | belushya guba
City not found. Skipping...
Processing Record 8 of Set 1 | talnakh
Processing Record 9 of Set 1 | urucara
Processing Record 10 of Set 1 | ust-kamchatsk
City not found. Skipping...
Processing Record 11 of Set 1 | jamestown
Processing Record 12 of Set 1 | oranjemund
Processing Record 13 of Set 1 | esperance
Processing Record 14 of Set 1 | port alfred
Processing Record 15 of Set 1 | barrow
Processing Record 16 of Set 1 | ponta do sol
Processing Record 17 of Set 1 | mahebourg
Processing Record 18 of Set 1 | lavrentiya
Processing Record 19 of Set 1 | nizhneyansk
City not found. Skipping...
Processing Record 20 of Set 1

Processing Record 40 of Set 4 | cameta
Processing Record 41 of Set 4 | ancud
Processing Record 42 of Set 4 | oxford
Processing Record 43 of Set 4 | inhambane
Processing Record 44 of Set 4 | laguna
Processing Record 45 of Set 4 | hovd
Processing Record 46 of Set 4 | gat
Processing Record 47 of Set 4 | ciudad bolivar
Processing Record 48 of Set 4 | kieta
Processing Record 49 of Set 4 | deputatskiy
Processing Record 50 of Set 4 | berlevag
Processing Record 1 of Set 5 | kijang
Processing Record 2 of Set 5 | santa vitoria
Processing Record 3 of Set 5 | cidreira
Processing Record 4 of Set 5 | puerto ayora
Processing Record 5 of Set 5 | cabedelo
Processing Record 6 of Set 5 | xuddur
Processing Record 7 of Set 5 | tidore
City not found. Skipping...
Processing Record 8 of Set 5 | lodja
Processing Record 9 of Set 5 | north bend
Processing Record 10 of Set 5 | saleaula
City not found. Skipping...
Processing Record 11 of Set 5 | coari
Processing Record 12 of Set 5 | upernavik
Processing Record 13 

Processing Record 24 of Set 8 | riyadh
Processing Record 25 of Set 8 | abu kamal
Processing Record 26 of Set 8 | lengshuijiang
Processing Record 27 of Set 8 | agirish
Processing Record 28 of Set 8 | pouebo
Processing Record 29 of Set 8 | eureka
Processing Record 30 of Set 8 | balabac
Processing Record 31 of Set 8 | umzimvubu
City not found. Skipping...
Processing Record 32 of Set 8 | lujan
Processing Record 33 of Set 8 | ibra
Processing Record 34 of Set 8 | wamba
Processing Record 35 of Set 8 | chernelytsya
Processing Record 36 of Set 8 | tigil
Processing Record 37 of Set 8 | the pas
Processing Record 38 of Set 8 | melchor de mencos
Processing Record 39 of Set 8 | methala
Processing Record 40 of Set 8 | prieska
Processing Record 41 of Set 8 | libreville
Processing Record 42 of Set 8 | clyde river
Processing Record 43 of Set 8 | huarmey
Processing Record 44 of Set 8 | salinas
Processing Record 45 of Set 8 | zhezkazgan
Processing Record 46 of Set 8 | gornyak
Processing Record 47 of Set 8

Processing Record 12 of Set 12 | miles city
Processing Record 13 of Set 12 | valdivia
Processing Record 14 of Set 12 | samoded
Processing Record 15 of Set 12 | eydhafushi
Processing Record 16 of Set 12 | pandhurna
Processing Record 17 of Set 12 | kaa-khem
Processing Record 18 of Set 12 | radebeul
Processing Record 19 of Set 12 | artyom
Processing Record 20 of Set 12 | ternate
Processing Record 21 of Set 12 | dongsheng
Processing Record 22 of Set 12 | toamasina
Processing Record 23 of Set 12 | weyburn
Processing Record 24 of Set 12 | plattsburgh
Processing Record 25 of Set 12 | marzuq
Processing Record 26 of Set 12 | iralaya
Processing Record 27 of Set 12 | fort nelson
Processing Record 28 of Set 12 | changji
Processing Record 29 of Set 12 | gizo
Processing Record 30 of Set 12 | puerto escondido
Processing Record 31 of Set 12 | hamilton
Processing Record 32 of Set 12 | kahului
Processing Record 33 of Set 12 | sembakung
Processing Record 34 of Set 12 | qasigiannguit
Processing Record 35 

In [19]:
len(city_data)

680

In [20]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Current Description
0,Wuwei,37.9281,102.6414,61.75,16,3,4.94,CN,2022-06-12 16:27:53,clear sky
1,Busselton,-33.6500,115.3333,58.66,69,100,31.79,AU,2022-06-12 16:27:53,overcast clouds
2,Punta Arenas,-53.1500,-70.9167,51.91,71,40,13.80,CL,2022-06-12 16:27:41,scattered clouds
3,Norman Wells,65.2820,-126.8329,42.84,93,100,14.97,CA,2022-06-12 16:27:54,light rain
4,Qaanaaq,77.4840,-69.3632,30.81,69,89,3.42,GL,2022-06-12 16:27:54,overcast clouds
5,Talnakh,69.4865,88.3972,63.21,44,100,5.50,RU,2022-06-12 16:27:54,overcast clouds
6,Urucara,-2.5364,-57.7600,82.04,80,52,1.41,BR,2022-06-12 16:27:55,heavy intensity rain
7,Jamestown,42.0970,-79.2353,69.39,57,99,1.01,US,2022-06-12 16:27:03,overcast clouds
8,Oranjemund,-28.5500,16.4333,61.38,66,100,9.51,NA,2022-06-12 16:27:55,overcast clouds
9,Esperance,-33.8667,121.9000,59.22,69,80,20.38,AU,2022-06-12 16:27:55,broken clouds


In [21]:
new_column_order = ["City", "Country", "Date", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed","Current Description"]
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Wuwei,CN,2022-06-12 16:27:53,37.9281,102.6414,61.75,16,3,4.94,clear sky
1,Busselton,AU,2022-06-12 16:27:53,-33.6500,115.3333,58.66,69,100,31.79,overcast clouds
2,Punta Arenas,CL,2022-06-12 16:27:41,-53.1500,-70.9167,51.91,71,40,13.80,scattered clouds
3,Norman Wells,CA,2022-06-12 16:27:54,65.2820,-126.8329,42.84,93,100,14.97,light rain
4,Qaanaaq,GL,2022-06-12 16:27:54,77.4840,-69.3632,30.81,69,89,3.42,overcast clouds
5,Talnakh,RU,2022-06-12 16:27:54,69.4865,88.3972,63.21,44,100,5.50,overcast clouds
6,Urucara,BR,2022-06-12 16:27:55,-2.5364,-57.7600,82.04,80,52,1.41,heavy intensity rain
7,Jamestown,US,2022-06-12 16:27:03,42.0970,-79.2353,69.39,57,99,1.01,overcast clouds
8,Oranjemund,NA,2022-06-12 16:27:55,-28.5500,16.4333,61.38,66,100,9.51,overcast clouds
9,Esperance,AU,2022-06-12 16:27:55,-33.8667,121.9000,59.22,69,80,20.38,broken clouds


In [22]:
# Create the output file (CSV).
output_data_file = "WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")
